In [47]:
import sqlite3

In [48]:
db_path = 'concerts.db'
conn = sqlite3.connect(db_path)

In [49]:
import pandas as pd

In [50]:
db_table_names = ('artist', 'customer', 'genre', 'performance', 'ticket', 'ticketpricing')
table_df_dict = dict()

for table_name in db_table_names:
    table_df_dict[table_name] = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)

In [ ]:
df = pd.DataFrame()

In [57]:
def populate_with_data(df):
    df['Ticket Price'] = table_df_dict['ticket']['ticketPrice']
    df['Ticket Category'] = table_df_dict['ticket']['type']

    merged_df = pd.merge(table_df_dict['artist'], table_df_dict['genre'], on='idGenre', how='left')
    merged_df = pd.merge(table_df_dict['performance'], merged_df, on='idArtist', how='left')
    merged_df = pd.merge(table_df_dict['ticket'], merged_df, on='idPerformance', how='left')

    df['Artist'] = merged_df['name_x']
    df['Genre'] = merged_df['genreName']
    df['Performance Name'] = merged_df['name_y']
    df['Performance Date'] = merged_df['date']

    df['Official Ticket Price'] = table_df_dict['ticket'].merge(
        table_df_dict['ticketpricing'][['idPerformance', 'Type', 'price']],
        left_on=['idPerformance', 'type'],
        right_on=['idPerformance', 'Type'],
        how='left'
    )['price']

In [58]:
populate_with_data(df)
df

,Artist,Genre,Performance Name,Performance Date,Official Ticket Price,Ticket Price,Ticket Category
0,ABX,Pop,ABBA,2025-06-18 20:30:00,2000,2000,A
1,Adelaire,Pop,Adele,2025-06-19 20:30:00,900,900,C
2,BamaBeats,Country,Alabama,2025-06-20 20:30:00,200,200,D
3,ChillSync,Rock,Coldplay,2025-06-21 20:30:00,1000,1000,B
4,Distorto,Metal,Disturbed,2025-06-25 21:30:00,900,900,A
5,Dafunk,Electronic,Daft,2025-06-26 22:30:00,450,450,D
6,Metallix,Metal,Metallica,2025-06-27 22:00:00,2000,2000,C
7,Q-Majesty,Rock,Queen,2025-06-28 18:30:00,3000,3000,B
8,Imaginix,Pop,Imagine,2025-07-04 21:00:00,950,950,A
9,Nirvex,Rock,Nirvana,2025-07-05 21:30:00,900,900,D
